In [8]:
import os
import numpy as np
from scipy.interpolate import interp1d


def calculate_delta_f(phi, free):    
    # Filter data based on conditions
    A = free[phi < 0]
    B = free[(phi > 0) & (phi < 2.2)]
    
    # Calculate free energies
    fesA = -2.49 * np.logaddexp.reduce(-1 / 2.49 * A)
    fesB = -2.49 * np.logaddexp.reduce(-1 / 2.49 * B)
    return fesB - fesA

def load_data(fes_dir, method):
    file = os.path.join(fes_dir, str(seed), "fes/100.dat")

    # Load data from the file
    data = np.loadtxt(file, comments='#')
    
    with open(file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    phi_idx = keys.index(cv_name[method])
    free_idx = keys.index('file.free')

    phi = data[:, phi_idx]
    free = data[:, free_idx]
    return phi, free

def load_interp(fes_dir, method):
    fes_file = os.path.join(fes_dir, str(seed), "fes/100.dat")
    cv_file = os.path.join(fes_dir, str(seed), "COLVAR")

    # Load data from the file
    data = np.loadtxt(fes_file, comments='#')
    
    with open(fes_file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    cv_idx = keys.index(cv_name[method])
    free_idx = keys.index('file.free')

    cv_grid = data[:, cv_idx]
    free_grid = data[:, free_idx]

    # Load data from the file
    data = np.loadtxt(cv_file, comments='#')
    
    with open(cv_file, 'r') as file:
        first_line = file.readline().strip()
        
    keys = first_line.split()[2:]
    cv_idx = keys.index(cv_name[method])
    phi_idx = keys.index('phi')

    cv = data[:, cv_idx]
    phi = data[:, phi_idx]

    # print(cv_grid)

    # 선형 보간: deep_cv 범위 밖이면 extrapolate
    fes_interp = interp1d(
        cv_grid, 
        free_grid, 
        kind='linear', 
        fill_value="extrapolate"
    )

    free = fes_interp(cv)
    return phi, free
    
ns = '10'

cv_name = {
    'ref': 'phi',
    'phi': 'phi',
    'AE': 'deep.node-0',
    'TAE': 'deep.node-0',
    'VDE': 'deep.node-0',
    'DeepTDA': 'deep.node-0',
    'DeepLDA': 'deep.node-0',
    'DeepTICA': 'deep.node-0',
}
method = 'ref'
# method = 'phi'
# method = 'AE'
# method = 'TAE'
# method = 'VDE'
# method = 'DeepLDA'
# method = 'DeepTDA'
# method = 'DeepTICA'

base_dir = f'/home/guest_sky/geodesic-interpolation-cv/simulations/aldp/{method}/{ns}/log'
date = sorted(os.listdir(base_dir))[-1]
fes_dir = os.path.join(base_dir, date)

delta_fs = []
# Iterate through all subdirectories
for seed in range(4):
    if method in ['phi','ref']:
        phi, free = load_data(fes_dir, method)
    else:
        phi, free = load_interp(fes_dir, method)
        
    delta_f = calculate_delta_f(phi, free)
    if np.any(np.isinf(delta_f)):
        continue
    delta_fs.append(delta_f)
    print(f"{delta_f}")

# Calculate mean and standard deviation of delta F
delta_f_mean = np.mean(delta_fs)
delta_f_std = np.std(delta_fs)

# Output results
print(f"{ns}ns Mean: {delta_f_mean}")
print(f"{ns}ns Std: {delta_f_std}")

21.505876690378386
-0.3710346492185863
6.338012352598298
8.537954551172788
10ns Mean: 9.002702236232722
10ns Std: 7.92957875954302
